In [1]:
model_size = 1.7
Q_size = 16

m = 1.2*(model_size*4)/(32/Q_size)

m

4.08

In [2]:
qwen1p7 = "Qwen/Qwen3-1.7B"
qwenp6 = "Qwen/Qwen3-0.6B"

## Pipeline

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
from transformers import pipeline

/home/miguel/miniforge3/envs/DataScience/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
qwen = pipeline("text-generation", model=qwenp6, device=1)

Device set to use cuda:1


RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
ans = qwen("Quem descobriu o Brasil?", max_new_tokens=100, do_sample=True, temperature=0.1)

In [8]:
ans

[{'generated_text': 'Quem descobriu o Brasil? Quem foi o primeiro colonizador? Quem foi o primeiro governador? Quem foi o primeiro presidente? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro presidente da República? Quem foi o primeiro'}]

## LangChain_HuggingFace

In [1]:
from langchain_huggingface.llms import HuggingFacePipeline

In [2]:
qwen1p7 = "Qwen/Qwen3-1.7B"
qwenp6 = "Qwen/Qwen3-0.6B"

In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:
torch.cuda.empty_cache()

In [ ]:
qwen1p7 = "Qwen/Qwen3-1.7B"
qwenp6 = "Qwen/Qwen3-0.6B" 
hf = HuggingFacePipeline.from_model_id(
    model_id=qwenp6,
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 400, "do_sample": True, "temperature": 0.1, "top_p": 0.7, "top_k": 50, "repetition_penalty": 1.2, 
                     "num_return_sequences": 1},
    device=0,
    
)

/home/miguel/miniforge3/envs/DataScience/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.07s/it]
Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 5.65 GiB of which 32.00 MiB is free. Including non-PyTorch memory, this process has 5.60 GiB memory in use. Of the allocated memory 5.43 GiB is allocated by PyTorch, and 93.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
from langchain_core.prompts import PromptTemplate

template = """Pergunta: {question}

Resposta: vamos pensar passo a passo."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf#.bind(skip_llm=True, skip_prompt=True, verbose=True, return_only_outputs=True, tags=["qwen"])
question = "mostre o passo a passo da separação de variaveis em EDP"

In [7]:
print(chain.invoke({"question": question}))

`generation_config` default values have been modified to match model-specific defaults: {'top_k': 20}. If this is not desired, please set these values explicitly.


Pergunta: mostre o passo a passo da separação de variaveis em EDP

Resposta: vamos pensar passo a passo. Primeiro, identificamos as equações e os valores iniciais para cada variável. Em seguida, isolamos as variáveis que estão relacionadas com a variação do estado (como temperatura ou pressão) e as que não são diretamente influenciadas por essas variações.

Explicando mais formalmente:

1. Identificar todas as equações diferenciais no problema.
2. Determinar os valores iniciais para cada variável.
3. Isolar as variáveis que dependem da mudança na sua função (variáveis dependentes).
4. Isolar as variáveis independentemente das mudanças nas funções (variáveis independentes).

Exemplo:
Se temos uma equação diferencial dA/dt = k*A(t), onde A é a quantidade de matéria e t é o tempo, então podemos determinar que A está dependendo da temperatura T(t). Porém, se teremos outra equação como dT/dt = -k*T + c, donde T é a temperatura, then we can determine that T is the variable depending on tempe

## AutoTokenizer

In [20]:
from transformers import AutoTokenizer
import torch

In [ ]:
model_qwen_16 = "Qwen/Qwen3-1.7B"

tokenizer = AutoTokenizer.from_pretrained(model_qwen_16) 
# carrega o tokenizador do modelo

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt") 
print(inputs)

{'input_ids': tensor([[    40,   3003,   1012,   8580,    369,    264,    472,  35268,  16281,
           3308,    847,   4361,   2272,     13],
        [    40,  12213,    419,    773,   1753,      0, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


transforma raw_inputs em tensores, primeiro divide raw_inputs em tokens, depois transforma os tokens em ids, e depois transforma os ids em tensores

Argumentos:

**padding:** é utlizado para deixar os inputs do mesmo tamanho

**truncation:** é utilizado para cortar os inputs que são maiores que o tamanho máximo do modelo

**max_length:** pode ser utilizado para definir o tamanho máximo dos tensores

**return_tensors:** é utilizado para retornar os tensores no formato especificado, nesse caso pytorch

In [22]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_qwen_16)#, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True)
outputs = model(**inputs)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.19s/it]


In [27]:
encoded = inputs['input_ids']
print(encoded)

tensor([[    40,   3003,   1012,   8580,    369,    264,    472,  35268,  16281,
           3308,    847,   4361,   2272,     13],
        [    40,  12213,    419,    773,   1753,      0, 151643, 151643, 151643,
         151643, 151643, 151643, 151643, 151643]])


In [29]:
model_inputs = torch.tensor(encoded) # converte para tensor

/tmp/ipykernel_3407116/1245321750.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model_inputs = torch.tensor(encoded) # converte para tensor


In [30]:
out = model(model_inputs)
print(out)

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 6.4866,  6.8487,  8.6039,  ...,  0.8221,  0.8221,  0.8221],
         [ 9.1542,  9.3961,  5.4054,  ...,  0.6275,  0.6275,  0.6275],
         [ 9.4493,  9.0522,  4.3740,  ...,  0.6883,  0.6884,  0.6883],
         ...,
         [12.5623, 10.7681,  7.3639,  ...,  1.3935,  1.3934,  1.3935],
         [23.5219, 15.9169, 12.8333,  ...,  1.6118,  1.6118,  1.6118],
         [10.1738,  5.5156,  8.8069,  ...,  1.0480,  1.0480,  1.0480]],

        [[ 6.4866,  6.8487,  8.6039,  ...,  0.8221,  0.8221,  0.8221],
         [13.4813, 13.4038,  9.8580,  ...,  1.0742,  1.0742,  1.0742],
         [18.0200, 13.0740,  8.9805,  ...,  1.5558,  1.5558,  1.5559],
         ...,
         [13.3215,  9.1971, 10.2730,  ...,  1.2982,  1.2982,  1.2981],
         [15.0423,  8.8453, 10.2103,  ...,  1.3820,  1.3820,  1.3820],
         [15.1075,  9.9669, 11.0436,  ...,  1.5256,  1.5256,  1.5256]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=DynamicCache(layers=